# Migdar-k8s Minikube quickstart

## Install Conda environment to run the Jupyter notebooks

Clone the repo and change to project directory

```
git clone git@github.com:hasadna/migdar-k8s.git
cd migdar-k8s
```

Create the Conda environment

```
conda env create -f notebooks/conda-environment.yaml
```

Activate the environment, install dependencies and start Jupyter

```
. activate migdar-k8s
pip install bash_kernel
python -m bash_kernel.install
jupyter lab
```

Open the quickstart notebook under the notebooks directory and continue this guide

## Install Minikube

Following command should work for Linux, for other OS, see [Minikube docs](https://github.com/kubernetes/minikube/blob/master/README.md#installation)

```
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64 &&\
sudo install minikube-linux-amd64 /usr/local/bin/minikube &&\
rm minikube-linux-amd64
```

### (Optional) Ensure a clean Minikube cluster

In [ ]:
minikube delete; rm -rf ~/.minikube

## Start Minikube

```
minikube start
```

## Define bash functions

In [27]:
source functions.bash

## Initialize the minikube environment

Optional - delete the host path used for persistent storage

```
minikube ssh -- sudo rm -rf /var/migdar-minikube-storage
```

Create the host path

```
minikube ssh -- sudo mkdir -p /var/migdar-minikube-storage
```

Make sure we are connected to the right cluster (should see single minikube node)

In [2]:
_env minikube
kubectl get nodes

minikube: connected
NAME       STATUS    ROLES     AGE       VERSION
minikube   Ready     master    8d        v1.10.0


## Install Helm

Install the client (requires sudo):

```
curl -L https://raw.githubusercontent.com/hasadna/hasadna-k8s/master/apps_travis_script.sh | bash /dev/stdin install_helm
```

Install the server:

In [5]:
_env minikube
helm init --history-max 2 --upgrade --wait
# Verify correct version is installed on both server and client
helm version

minikube: connected
$HELM_HOME has been configured at /home/ori/.helm.

Tiller (the Helm server-side component) has been upgraded to the current version.
Happy Helming!
Client: &version.Version{SemVer:"v2.8.2", GitCommit:"a80231648a1473929271764b920a8e346f6de844", GitTreeState:"clean"}
Server: &version.Version{SemVer:"v2.8.2", GitCommit:"a80231648a1473929271764b920a8e346f6de844", GitTreeState:"clean"}


## Deploy

#### Infrastructure

In [2]:
_env minikube
helm_upgrade_external_chart elasticsearch --install && get_pod_status_log -l app=elasticsearch &&\
helm_upgrade_external_chart postgres --install && get_pod_status_log -l app=postgres

minikube: connected
Switched to context "minikube".
RELEASE_NAME=migdar-elasticsearch-minikube
CHART_DIRECTORY=charts-external/elasticsearch
Release "migdar-elasticsearch-minikube" has been upgraded. Happy Helming!
LAST DEPLOYED: Thu Oct 18 22:33:39 2018
NAMESPACE: minikube
STATUS: DEPLOYED

RESOURCES:
==> v1/Service
NAME           TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)            AGE
elasticsearch  ClusterIP  10.99.159.148  <none>       9200/TCP,9300/TCP  4h

==> v1beta1/Deployment
NAME           DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
elasticsearch  1        1        1           1          1h

==> v1/Pod(related)
NAME                           READY  STATUS   RESTARTS  AGE
elasticsearch-5b6dfcc96-8llsj  1/1    Running  0         1h


Great Success!
waiting for pod...
elasticsearch pod is running
conditions:
- lastProbeTime: null
  lastTransitionTime: '2018-10-18T18:00:14Z'
  status: 'True'
  type: Initialized
- lastProbeTime: null
  lastTransitionTime: '2018-10-18T18:00:

In [3]:
get_pod_status_log -l app=postgres

conditions:
- lastProbeTime: null
  lastTransitionTime: '2018-10-18T19:33:47Z'
  status: 'True'
  type: Initialized
- lastProbeTime: null
  lastTransitionTime: '2018-10-18T19:33:52Z'
  status: 'True'
  type: Ready
- lastProbeTime: null
  lastTransitionTime: '2018-10-18T19:33:47Z'
  status: 'True'
  type: PodScheduled
containerStatuses:
- containerID: docker://d55d66a4ea07d257d27ed012fe831a16ceb1ab8ea63f0285c5458cd393e09d91
  image: budgetkey/budgetkey-postgres:3efcf352910056cd965ca9a0212722fa03fb1840
  imageID: docker-pullable://budgetkey/budgetkey-postgres@sha256:4d6fc25962c6318d8ef33c492f53c972c5366c80ba9a27cc33ff9acda234617d
  lastState:
    terminated:
      containerID: docker://8fb29f9aa585d91813548c1e45affe79a9a887ecd359551c13f89086f4469d19
      exitCode: 3
      finishedAt: '2018-10-18T19:33:51Z'
      reason: Error
      startedAt: '2018-10-18T19:33:48Z'
  name: postgres
  ready: true
  restartCount: 1
  state:
    running:
      startedAt: '2018-10-18T19:33:52Z'
hostIP: 10.0

#### Apis

In [4]:
source functions.bash
_env minikube
helm_upgrade_external_chart data-api --install && get_pod_status_log -l app=data-api &&\
helm_upgrade_external_chart search-api --install && get_pod_status_log -l app=search-api

minikube: connected
Switched to context "minikube".
RELEASE_NAME=migdar-data-api-minikube
CHART_DIRECTORY=charts-external/data-api
Release "migdar-data-api-minikube" has been upgraded. Happy Helming!
LAST DEPLOYED: Thu Oct 18 22:34:30 2018
NAMESPACE: minikube
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME      DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
data-api  1        1        1           1          9m

==> v1/Pod(related)
NAME                       READY  STATUS   RESTARTS  AGE
data-api-84dd7487cb-8hs8x  1/1    Running  6         9m

==> v1/Service
NAME                  TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)   AGE
open-budget-data-api  ClusterIP  10.111.203.174  <none>       8000/TCP  9m


Great Success!
waiting for pod...
data-api pod is running
conditions:
- lastProbeTime: null
  lastTransitionTime: '2018-10-18T19:24:39Z'
  status: 'True'
  type: Initialized
- lastProbeTime: null
  lastTransitionTime: '2018-10-18T19:34:03Z'
  status: 'True'
  type: Ready
- la

In [7]:
_env minikube
kubectl get pods

minikube: connected
NAME                            READY     STATUS    RESTARTS   AGE
data-api-84dd7487cb-8hs8x       1/1       Running   6          11m
elasticsearch-5b6dfcc96-8llsj   1/1       Running   0          1h
postgres-db7f7c79b-5676c        1/1       Running   1          1m
search-api-69f866887b-q4ws9     1/1       Running   0          7m
